<a href="https://colab.research.google.com/github/Zeilion/Covid_dashboard/blob/main/covid_dashboard_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# My COVID-19 Dashboard - v0.2.0
***
Author: Matthieu PELINGRE  
Date : 18/02/2022

Description:
Vous pourriez interpréter ces données pour créer un dashbord dans lequel 
on peut choisir :

- la donnée à laquelle on s'intéresse (confirmed, deaths ou recovered)
- le pays auquel on s'intéresse (idéalement dans une liste triée)
- la période

et en fonction, afficher deux courbes qui montrent sur cette période :

- la donnée brute (une fonction croissante donc)
- sa dérivée (la différence avec le jour précédent)


***
***
## Core

In [1]:
# if needed, remove the '#' before the next line to install it
#!pip install ipympl

***
### Modules

In [2]:
# DataMining :
# pour aller chercher l'URL
import requests
# pour charger le JSON en objets Python
import json

# Traitement :
import numpy as np
import pandas as pd

# Affichage :
%matplotlib inline
# magic must be inline for Google Colab, widget for JupyterLab
import matplotlib.pyplot as plt
from IPython.display import display

# pour permettre l'utilisation des widgets sur Google Colab :
from google.colab import output
output.enable_custom_widget_manager()

# Widgets :
import ipywidgets as widgets

# pour installer le drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


***
### Importation des données

In [3]:
# repo github du CSSE de l'Université Johns Hopkins (brut) :
official_url = "https://github.com/CSSEGISandData/COVID-19"
# repo github qui expose les données dans un seul fichier au jormat JSON :
abridged_url = "https://pomber.github.io/covid19/timeseries.json"

# importation
url = abridged_url
req = requests.get(url)
## un peu de vérification
# si req.ok n'est pas True, il y a un souci 
# avec le réseau ou cette URL
verified = req.ok
if verified:
    # en utilisant la property `text` on décode en Unicode
    encoded = req.text
    # que l'on peut ensuite décoder en json
    decoded = json.loads(encoded)
    # decoded est un dict {pays: données}
else:
    raise ImportError(f"Can't load requested file from {url}")
    #print(f"404 error - Can't load requested file from {url}")

***
### Fonction de création de la DataFrame Globale

In [4]:
def decoded_to_dataframe(decoded):
    # création du dictionnaire temporaire
    dict_df = {'World': np.zeros((len(decoded["US"]), 3))}
    for countryname in decoded.keys():
        # extraction données du pays
        country = decoded[countryname]
        # création de la DataFrame
        df_temp = pd.DataFrame(country)
        # date en index
        df_temp = df_temp.set_index('date')
        # transforme l'index des dates en DatetimeIndex
        df_temp.index = pd.to_datetime(df_temp.index)
        # dictionnaire {pays: dataframe}
        dict_df[countryname] = df_temp
        # ajout des valeurs à world
        dict_df['World'] += df_temp
    
    # concaténation des dataframes du dictionnaire
    df_res = pd.concat(dict_df.values(), keys=dict_df.keys(), names=['country'], axis=1)
    # renome le nom du multiindex des données
    df_res.columns = df_res.columns.set_names('data', level=1)
    
    # test d'intégrité
    if np.any(np.isnan(df_res)):
        df_res = df_res.fillna(0)  # remplace les NaN par 0 s'il y en a
    
    return df_res

Fonctions de sauvegarde/chargement des dataframes au format json :

In [5]:
def save_covid_json(decoded):
    df_s = decoded_to_dataframe(decoded)
    df_s.to_json('FullDataCovid.json')
    
def load_covid_json(file='FullDataCovid.json'):
    df_loaded = pd.read_json(file)  # charge le json
    df_loaded.index.name = 'date'  # change le nom de l'index
    # restore le multiindex :
    df_loaded.columns = pd.MultiIndex.from_tuples([eval(x) for x in df_loaded.columns],
                                                  names=['country', 'data'])
    return df_loaded

Core : création de la dataframe ou chargement.

In [6]:
if not(verified):
    # si le fichier de base n'a pas pu être téléchargé, on charge notre json
    df = load_covid_json()
else:
    df = decoded_to_dataframe(decoded)

***
### Construction du tableau de bord

In [7]:
# contôle de la taille des widgets
l_100 = widgets.Layout(width='100%')
l_75 = widgets.Layout(width='73%')
l_66 = widgets.Layout(width='64%')
l_50 = widgets.Layout(width='48%')
l_33 = widgets.Layout(width='31%')
l_25 = widgets.Layout(width='23%')
l_style = {'description_width': 'initial'}  # permet de compter la descritption dans la taille
l_spacebtw = widgets.Layout(justify_content='space-between')  # espace les éléments entre eux
l_center = widgets.Layout(justify_content='center')

w_countryname = widgets.Dropdown(options=sorted(set(df.columns.get_level_values("country"))),
                                 value='World',
                                 description="Country",
                                 style=l_style,
                                 layout=l_33)

w_days = widgets.IntRangeSlider(value=[0, len(df) - 1],
                               min=0,
                               max=len(df) - 1,
                               step=1, 
                               description='Period (days)', 
                               continuous_update=False, 
                               style=l_style, 
                               layout=l_100)



w_data = widgets.Dropdown(options={"-" : "",
                                   "Confirmed cases" : "confirmed",
                                   "Deaths" : "deaths",
                                   "Recovered" : "recovered",
                                  },
                          value="",
                          description="Data",
                          style=l_style,
                          layout = l_33)

w_ddaily = widgets.Dropdown(options={"Both" : "",
                                     "Sum" : 1,
                                     "Daily" : 2,
                                    },
                            value="",
                            description="Curves (Data must be set)",
                            style=l_style,
                            layout = l_33)
    
# contruction du tableau de bord
dashboard = widgets.VBox([widgets.HBox([w_countryname, w_data, w_ddaily], layout=l_spacebtw),
                          widgets.HBox([w_days]),
                         ])

# dictionnaire de résultat pour interactive_output
def my_dashboard_country():
    return dict(countryname=w_countryname, data=w_data, days=w_days, ddaily=w_ddaily)

Bouton pour sauvegarder la DataFrame au format json :

In [8]:
save_data_button = widgets.Button(description='Save DataFrame',
                                  button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                  tooltip='Save the current DataFrame in ./FullDataCovid.json',
                                  icon='save')

# configure la sortie des widgets
#output = widgets.Output()
# le résultat d'une fonction peut être capté en 'output' grâce au décorateur "@output.capture()"
# ou "with output"

def on_save_button_clicked(b):
    save_covid_json(decoded)
    b.icon="check"
    b.description='DataFrame Saved'
    b.button_style='success'
    b.tooltip='DataFrame has been saved in ./FullDataCovid.json'


save_data_button.on_click(on_save_button_clicked)

***
### Fonction d'affichage

In [9]:
# interactively call country_last_days
# fonction d'affichage pour interactive_output :
def graph_plot(countryname, data, days, ddaily):
    global df
    # obligé d'utiliser df en global pour les widgets
    
    # ferme toutes les instances de pyplot avant d'en ouvrir une nouvelle
    plt.close('all')
    
    # création d'une dataframe temporaire, une deep copy
    df_temp = (df.loc[:, (countryname)]).copy()
    
    # létalité globale
    g_letality = df.loc[:, (countryname, 'deaths')].sum() / df.loc[:, (countryname, 'confirmed')].sum()
    # calcul de la létalité sur la période considérée
    p_letality = ((df_temp.iloc[days[0]:days[1]+1]).loc[:, 'deaths'].sum() 
                  / (df_temp.iloc[days[0]:days[1]+1]).loc[:, 'confirmed'].sum())
    
    # extraction des valeurs en premier
    # pour une certaine colonne, si spécifié
    if data:
        # extrait la série de la colonne voulue
        s = df_temp.loc[:, data]
        # applique un shift et remplace les NaN par 0
        s_shifted = ((df_temp.loc[:, data]).shift(1)).fillna(0)
        # pour créer la colonne daily
        df_temp['daily'] = s - s_shifted
        # extrait la colonne désirée et la colonne daily
        df_temp = df_temp.loc[:, [data, 'daily']]
        if ddaily:
            df_temp = df_temp.iloc[:, ddaily-1]
    
    # extrait les 'days' derniers jours
    df_temp = df_temp.iloc[days[0]:days[1]+1]
    
    
    # création de la figure et du subplot
    fig, ax = plt.subplots()
    # éfface l'entête du widget "Figure X"
    fig.canvas.header_visible = False
    # trace la dataframe dans le subplot
    df_temp.plot(ax=ax)
    # titre et y label :
    if data:
        plt.title(f'All {data} in {countryname} between \n'
                  f'{str(df.index[days[0]])[:-9]} and {str(df.index[days[1]])[:-9]}')
        #plt.ylabel(f'number of {data}')
    else:
        plt.title(f'All data in {countryname} between \n'
                  f'{str(df.index[days[0]])[:-9]} and {str(df.index[days[1]])[:-9]}')
        #plt.ylabel(f'numbers')
    # affiche une grille
    plt.grid(True)
    # déplace le label de la date à droite du graph
    ax.xaxis.set_label_coords(1.06, -0.025)
    #affichage létalité:
    ax.annotate(f'Global letality: {g_letality:.2%}\nOver the period: {p_letality:.2%}',
                xy=(.38, .78), xycoords='figure fraction')
    
    # affiche l'image finale
    plt.show();

In [10]:
# interaction entre les widgets et la fonction d'affichage
out = widgets.interactive_output(graph_plot, my_dashboard_country())

# construction du dashboard final
full_dashboard = (widgets.VBox([dashboard,
                                widgets.HBox([out], layout=l_center),
                                save_data_button
                               ]))

***
***
## Affichage

In [11]:
display(full_dashboard)

***
## tests

opérations :

In [12]:
df.loc[:, ('US', 'deaths')].sum() / df.loc[:, ('US', 'confirmed')].sum()  # létalité totale

0.017089579489107855

In [13]:
df.loc[:, (slice(None), 'confirmed')].tail(3)  # pour accéder uniquement aux confirmed de tous les pays

country,World,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,China,Colombia,...,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Sudan,Spain,Sri Lanka,Sudan,Summer Olympics 2020,Suriname,Sweden,Switzerland,Syria,Taiwan*,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Tonga,Trinidad and Tobago,Tunisia,Turkey,US,Uganda,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
data,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,...,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed,confirmed
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-02-15,415293249.0,171422,268491,262570,37361,98555,11,7342,8766174,408381,2969890,2324406,752605,32972,473462,1919102,52310,843717,3460301,55653,26552,7916,886108,365589,260491,27677468,24659,1049543,20725,550824,37923,55846,124139,118675,3208795,14187,7216,2677692,126374,6026988,...,497997,1891646,858670,4910,26260,3645269,16900,10707286,630599,59903,865,77334,2409081,2607052,53011,19666,17780,33549,2639062,21877,36693,141,121271,968393,13079683,78036352,162869,4785138,870358,18521452,788676,234063,7,506977,2572087,618527,505,11699,309870,231603
2022-02-16,417731955.0,171519,268940,262994,37452,98568,11,7395,8783208,410155,2967316,2362662,759753,33005,477750,1923031,52637,851636,3473015,55803,26567,8297,887089,366450,260491,27819996,25584,1054566,20729,553564,37947,55856,124343,118675,3216972,14187,7216,2709854,127248,6031130,...,514880,1920150,863461,5043,26260,3648968,16903,10744394,631816,59939,865,77450,2414463,2628093,53148,19732,17783,33549,2656411,22047,36704,141,121271,971460,13173859,78172840,162932,4818112,871315,18575733,795316,234359,7,508042,2606824,622175,507,11707,310155,232213
2022-02-17,419694923.0,171673,269301,263369,37522,98585,11,7395,8799858,411878,2990045,2393576,764202,33005,481512,1926570,52909,859884,3484518,55975,26567,8683,887089,367025,261913,27940119,27599,1059192,20743,556256,37947,55856,124787,119107,3224226,14187,7216,2747552,127796,6035143,...,533425,1943101,868142,5565,26260,3652024,16903,10778607,633051,59939,865,77549,2418560,2628093,53278,19797,17783,33549,2674477,22138,36710,210,122093,974214,13266265,78269443,162932,4853339,872210,18628702,800833,234600,7,508968,2643024,625028,507,11718,310474,232598


In [14]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=1, description='Slider', max=5, min=-5)